# 1. Data Collection

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
panel = soup.find('div', class_='mw-parser-output')

# 2. Data Formation

## 2.1 Data Preprocess

In [4]:
# Set up empty lists to save the results
PostalCodes_list = []
Boroughs_list = []
Neigborhoods_list = []
#Scan segment with 'p' in the the table
for Block in panel.find('table').find_all('p'):
    #Set up empty temporary variables for data preprocessing
    PostalCode = ''
    Borough = ''
    Neigborhoods = ''
    Neigborhood = ''
    #Fetch the Postal Code
    PostalCode = Block.b.text
    #Fetch all the cities with the same Postal Code
    Cities = Block.span.text
    try:
        #Extract the first city that is the Borough
        Borough = Cities.split("(")[0].strip()
        #Extract the Neigborhoods near their Borough
        if Borough != 'Not assigned':
            #print(Borough)
            Neigborhoods = Cities.split("(")[1].split(")")[0].strip().split('/')
            Neigborhoods = [string.strip() for string in Neigborhoods]
            Neigborhood = ", ".join(Neigborhoods)
            # Appending the Postal Code, Borough and Neigborhood in their lists
            PostalCodes_list.append(PostalCode)
            Boroughs_list.append(Borough)
            Neigborhoods_list.append(Neigborhood)
    except Exception as e:
        pass

## 2.2 Dataframe Creation

In [5]:
dict = {'PostalCode': PostalCodes_list, 'Borough': Boroughs_list, 'Neigborhoods': Neigborhoods_list}
#print(len(PostalCodes_list))
#print(len(Boroughs_list))
#print(len(Neigborhoods_list))
#print(PostalCodes_list[-1])
#print(Boroughs_list[-1])
#print(Neigborhoods_list[-1])
df = pd.DataFrame(dict)

# 3. Locations

## 3.1 Download locations

In [6]:
#!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data

In [7]:
df_locations = pd.read_csv('Geospatial_Coordinates.csv')

In [8]:
df_locations.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_locations.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 3.2 Combining dataframes

In [9]:
df.head()

,PostalCode,Borough,Neigborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M9A,Etobicoke,Islington Avenue


In [10]:
df_merge = df.set_index('PostalCode').join(df_locations.set_index('PostalCode'))
df_merge.head()

,Borough,Neigborhoods,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


In [15]:
df_merge = df_merge.reset_index()
df_merge.head()

,index,PostalCode,Borough,Neigborhoods,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


# 4. Clustering

## 4.1 Explore Neighborhoods

In [36]:
# Grouping the neighborhoods of Toronto
Toronto_data = df_merge[df_merge['Borough'] == 'Downtown Toronto'].drop('index', 1)
Toronto_data.head()

,PostalCode,Borough,Neigborhoods,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


### Let's check the size of the neigborhoods

In [38]:
Toronto_data.groupby('Borough').count()

,PostalCode,Neigborhoods,Latitude,Longitude
Borough,,,,
Downtown Toronto,17,17,17,17


In [39]:
Toronto_data

,PostalCode,Borough,Neigborhoods,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
23,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
24,M6G,Downtown Toronto,Christie,43.669542,-79.422564
29,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
35,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
41,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
47,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [43]:
neighborhoods = Toronto_data.drop(['PostalCode', 'Borough'], axis=1)

In [44]:
neighborhoods.head()

,Neigborhoods,Latitude,Longitude
2,"Regent Park, Harbourfront",43.654260,-79.360636
8,"Garden District, Ryerson",43.657162,-79.378937
14,St. James Town,43.651494,-79.375418
19,Berczy Park,43.644771,-79.373306
23,Central Bay Street,43.657952,-79.387383


## 4.2 Cluster neighborhoods

Run *k*-means to cluster the neighborhood into 6 clusters.

In [62]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 6

toronto_grouped_clustering = neighborhoods.drop('Neigborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 2, 5, 3, 1, 0, 3])

In [69]:
# add clustering labels
Toronto_data.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_data

,Cluster Labels,PostalCode,Borough,Neigborhoods,Latitude,Longitude
2,1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
8,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
23,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
24,4,M6G,Downtown Toronto,Christie,43.669542,-79.422564
29,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
35,0,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
41,0,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
47,0,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [71]:
Toronto_data.rename(columns={'Neigborhoods':'Neighborhoods'}, inplace=True)

## 4.3 Visualization

### Use geopy to retrieve the latitude and longitude of Downtown Toronto

In [78]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.geocoders
geopy.geocoders.options.default_timeout = 20
address = 'Downtown Toronto'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [80]:
import folium
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhoods'], Toronto_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

#### Cluster 1

In [84]:
Toronto_data.loc[Toronto_data['Cluster Labels']==0]

,Cluster Labels,PostalCode,Borough,Neighborhoods,Latitude,Longitude
8,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
23,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
29,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
35,0,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
41,0,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
47,0,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
96,0,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280


#### Cluster 2

In [85]:
Toronto_data.loc[Toronto_data['Cluster Labels']==1]

,Cluster Labels,PostalCode,Borough,Neighborhoods,Latitude,Longitude
2,1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
95,1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675


Cluster 3

In [86]:
Toronto_data.loc[Toronto_data['Cluster Labels']==2]

,Cluster Labels,PostalCode,Borough,Neighborhoods,Latitude,Longitude
79,2,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
83,2,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049


Cluster 4

In [87]:
Toronto_data.loc[Toronto_data['Cluster Labels']==3]

,Cluster Labels,PostalCode,Borough,Neighborhoods,Latitude,Longitude
90,3,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
98,3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160


Cluster 5

In [88]:
Toronto_data.loc[Toronto_data['Cluster Labels']==4]

,Cluster Labels,PostalCode,Borough,Neighborhoods,Latitude,Longitude
24,4,M6G,Downtown Toronto,Christie,43.669542,-79.422564


Cluster 6

In [89]:
Toronto_data.loc[Toronto_data['Cluster Labels']==5]

,Cluster Labels,PostalCode,Borough,Neighborhoods,Latitude,Longitude
86,5,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.39442
